# Projeto 1 - Ciência dos Dados

Nome: Henrique Rocha Bomfim

Nome: Nicholas Morales Balkins

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [6]:


import pandas as pd

import numpy as np
import os
import re

In [7]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

os.chdir("..\data")

Esperamos trabalhar no diretório
c:\Users\nicom\OneDrive\Documentos\Projeto 1 - cdados\22-2a-cd-p1-grupo_henriquerb1\notebooks


Carregando a base de dados com as notícias classificadas manualmente:

In [8]:
filename = 'dados.xlsx'

In [9]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Titulo,Descrição,Pagina,Target
0,Ucrânia oferece recompensa “generosa” para que...,A Ucrânia está de olho nas carteiras de cripto...,68,0
1,"Só 12,9% da oferta disponível de bitcoin no me...",Conforme o preço do bitcoin (BTC) continua sub...,100,2
2,Presidente do Paraguai veta lei que regula as ...,"O presidente do Paraguai, Mario Abdo Benítez, ...",4,2
3,Corretora de criptomoedas Coinbase vai entrar ...,"A Coinbase, a maior corretora de criptomoedas ...",5,0
4,Por que criptomoedas e esports estão cada vez ...,Quando a corretora de criptomoedas FTX anuncio...,13,1


In [10]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Titulo,Descrição,Pagina,Target
0,Binance libera compra de oito criptomoedas com...,A Binance liberou oito criptomoedas que agora ...,73,1
1,Veja qual criptomoeda pode “herdar” os minerad...,"Holders de ETC, a criptomoeda da rede Ethereum...",5,0
2,Venezuela vai punir quem negociar criptomoedas...,Os venezuelanos que negociam criptomoedas em e...,36,2
3,Corretora de criptomoedas desiste de patrocina...,A corretora Crypto.com desistiu de um enorme a...,3,2
4,Golpistas lançam falsa criptomoeda da empresa-...,Nos últimos dias começou a circular no mercado...,71,2


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o que considerou como relevante ou não relevante na classificação das notícias (Target).

Criptomoedas -> Se foi aderida para algum processo ou não, e se não se aplica à notícia.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [13]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    #text_subbed = re.sub('(\s+)(de|em|do|para)(\s+)','\1\3',text_subbed)
    return text_subbed

train_0 = train.loc[train["Target"] == 0,:]
train_1 = train.loc[train["Target"] == 1,:]
train_2 = train.loc[train["Target"] == 2,:]

train_0_titulos = ''
for palavra in train_0.Titulo:
    train_0_titulos += palavra
train_0_titulos = cleanup(train_0_titulos).split()
train_0_titulos

train_1_titulos = ''
for palavra in train_1.Titulo:
    train_1_titulos += palavra
train_1_titulos = cleanup(train_1_titulos).split()


train_2_titulos = ''
for palavra in train_2.Titulo:
    train_2_titulos += palavra
train_2_titulos = cleanup(train_2_titulos).split()



serie_train_0_titulos = pd.Series(train_0_titulos)
serie_train_1_titulos = pd.Series(train_1_titulos)
serie_train_2_titulos = pd.Series(train_2_titulos)

print(serie_train_0_titulos.value_counts(),"\n")
print(serie_train_1_titulos.value_counts(),'\n')
print(serie_train_2_titulos.value_counts())

de                   228
criptomoedas         136
para                  67
e                     67
do                    64
                    ... 
novas                  1
criptomoedasPreso      1
PF                     1
tráfico                1
Plenário               1
Length: 1208, dtype: int64 

de                       70
criptomoedas             52
e                        36
em                       32
do                       26
                         ..
Libertadores              1
patrocinar                1
criptomoedasCorretora     1
pagar                     1
baixa                     1
Length: 551, dtype: int64 

de               54
criptomoedas     33
a                18
em               17
e                15
                 ..
desativados       1
eletrônicos       1
milhõesCaixas     1
350               1
2010              1
Length: 393, dtype: int64


In [14]:
tabela_absoluto_train_0 = serie_train_0_titulos.value_counts()
tabela_absoluto_train_1 = serie_train_1_titulos.value_counts()
tabela_absoluto_train_2 = serie_train_2_titulos.value_counts()

tabela_relativa_train_0 = serie_train_0_titulos.value_counts(True)
tabela_relativa_train_1 = serie_train_1_titulos.value_counts(True)
tabela_relativa_train_2 = serie_train_2_titulos.value_counts(True)
print(tabela_relativa_train_0,"\n")
print(tabela_relativa_train_1,"\n")
print(tabela_relativa_train_2)

de                   0.082133
criptomoedas         0.048991
para                 0.024135
e                    0.024135
do                   0.023055
                       ...   
novas                0.000360
criptomoedasPreso    0.000360
PF                   0.000360
tráfico              0.000360
Plenário             0.000360
Length: 1208, dtype: float64 

de                       0.064397
criptomoedas             0.047838
e                        0.033119
em                       0.029439
do                       0.023919
                           ...   
Libertadores             0.000920
patrocinar               0.000920
criptomoedasCorretora    0.000920
pagar                    0.000920
baixa                    0.000920
Length: 551, dtype: float64 

de               0.074896
criptomoedas     0.045770
a                0.024965
em               0.023578
e                0.020804
                   ...   
desativados      0.001387
eletrônicos      0.001387
milhõesCaixas    0.001387
3

In [15]:
portugues = train_0_titulos + train_1_titulos + train_2_titulos
serie_portugues = pd.Series(portugues)
tabela_portugues_relativa = serie_portugues.value_counts
portugues_soma = tabela_absoluto_train_1.sum() + tabela_absoluto_train_0.sum()
prob0 = tabela_absoluto_train_0.sum()/portugues_soma
prob1 = tabela_absoluto_train_1.sum()/portugues_soma
prob2 = tabela_absoluto_train_2.sum()/portugues_soma
print(prob0,"\n",prob1, '\n', prob2)
print(prob0 + prob1 + prob2)

0.7186124773492104 
 0.28138752265078953 
 0.18664250582448874
1.1866425058244887


In [ ]:
probFraseDado0 = 1
probFraseDado1 = 1
prob_frase = tabela_portugues_relativa
frase_lista = frase.split()

for palavra in frase_lista:
    probFraseDado1 = probFraseDado1 * tabela_relativa_train_1[palavra]
    probFraseDado0 = probFraseDado0 * tabela_relativa_train_0[palavra]
print(probFraseDado1)
print(probFraseDado0)

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das notícias entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas notícias. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas caterogias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por caterogia (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**